# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [1]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

In [9]:
from geopandas import GeoSeries, GeoDataFrame

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [2]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')

Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [3]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [4]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,24,14,,72,,2017-02-21 09:01:11 AM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,2,31,,79,,2017-02-21 09:01:56 AM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,7,19,,82,,2017-02-21 09:00:53 AM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,30,31,,83,,2017-02-21 09:02:13 AM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,18,21,,116,,2017-02-21 09:02:09 AM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [5]:
# there is one row for each bike station.  How many stations are there? 
len(df)

665

In [10]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

(-74.096936600000006,
 40.661063371900603,
 -73.929891100000006,
 40.804212999999997)

In [11]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

(957370.14732175611,
 180120.27032614074,
 1003695.8507545569,
 232275.23054640222)

In [15]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

366 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [17]:
brooklyn = boros.geometry[1]
in_br = df.geometry.within(brooklyn)
print(sum(in_br), 'stations in Brooklyn')

236 stations in Brooklyn


In [18]:
queens = boros.geometry[2]
in_qn = df.geometry.within(queens)
print(sum(in_qn), 'stations in Queens')

13 stations in Queens


In [19]:
staten = boros.geometry[0]
in_st = df.geometry.within(staten)
print(sum(in_st), 'stations in Staten Island')

0 stations in Staten Island


In [20]:
bronx = boros.geometry[4]
in_bx = df.geometry.within(bronx)
print(sum(in_bx), 'stations in the Bronx')

0 stations in the Bronx


In [28]:
print(sum(df['availableBikes'][in_mn]), 'available bikes in Manhattan')

3757 available bikes in Manhattan


In [29]:
print(sum(df['availableBikes'][in_br]), 'available bikes in Bronx')

2275 available bikes in Bronx


In [31]:
print(sum(df['availableBikes'][in_qn]), 'available bikes in Queens')

143 available bikes in Queens


In [32]:
print(sum(df['availableBikes'][in_st]), 'available bikes in Staten Island')

0 available bikes in Staten Island


In [33]:
print(sum(df['availableBikes'][in_bx]), 'available bikes in the Bronx')

0 available bikes in the Bronx
